In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.helpers import launch_env, wrap_env, view_results_ipython, change_exercise, seedall, force_done, evaluate_policy
from utils.helpers import SteeringToWheelVelWrapper, ResizeWrapper, ImgWrapper

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

INFO:aido-protocols:aido-protocols 5.0.5
22:02:51|zn|__init__.py:6|<module>(): zn 2.0.3
22:02:52|zj|__init__.py:5|<module>(): zj 2.0.4
22:02:52|gym-duckietown|__init__.py:10|<module>(): gym-duckietown 5.0.3

22:02:52|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-small_loop_cw-v0
22:02:52|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-straight_road-v0
22:02:52|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-loop_dyn_duckiebots-v0
22:02:52|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-loop_obstacles-v0
22:02:52|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-zigzag_dists-v0
22:02:52|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-udem1-v0
22:02:52|gym-duckietown|__init__.py:24|reg_map_env(): Registering gym environment id: Duckietown-small_loop-v0

# Reinforcement Learning Basics

Reinforcement Learning, as we saw in lecture, is the idea of learning a _policy_ in order to maximize future (potentially discounted) rewards. Our policy, similar to the imitation learning network, maps raw image observations to wheel velocities, and at every timestep, receives a _reward_ from the environment. 

Rewards can be sparse (`1` if goal or task is completed, `0` otherwise) or dense; in general, dense rewards make it easier to learn policies, but as we'll see later in this exercise, defining the correct dense reward is an engineering challenge on its own.

Today's reinforcement learning algorithms are often a mix between _value-based_ and _policy-gradient_ algorithms, instances of what is called an _actor-critic_ formulation. Actor-critic methods have had a lot of research done on them in recent years (especially within in the deep reinforcement learning era), and later in this exercise, we shall also rediscover the formulation's original problems and different methods currently used to stabilize learning.

We begin by defining two networks, an `Actor` and `Critic`; in this exercise, we'll be using a deep RL algorithm titled _Deep Deterministic Policy Gradients_. 

## Reward Engineering

In this part of the exercise, we will experiment with the reward formulation. Given the same model, we'll see how the effect of various reward functions changes the final policy trained. 

In the section below, we'll take a look at the reward function implemented in `gym-duckietown` with a slightly modified training loop. Traditionally, we `reset()` the environment to start an episode, and then `step()` the environment forward for a set amount of time, executing a new action. If this sounds a bit odd, especially for roboticists, you're right - in real robotics, most code runs asynchronously. As a result, although `gym-duckietown` runs locally by stopping the environment, the `AIDO` submissions will run asynchronously, executing the same action until a new one is received.

In [2]:
def updated_reward(env, c1=1.0, c2=-10, c3=40):
    # Compute the collision avoidance penalty
    pos, angle, speed = env.cur_pos, env.cur_angle, env.speed
    col_penalty = env.proximity_penalty2(pos, angle)
    
    # Get the position relative to the right lane tangent
    try:
        lp = env.get_lane_pos2(pos, angle)
    except NotInLane:
        reward = c3 * col_penalty
    else:
        # Compute the reward
        reward = (
                c1 * speed * lp.dot_dir +
                c2 * np.abs(lp.dist) +
                c3 * col_penalty
        )
    return reward

In [3]:
nepisodes = 3

In [4]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

class Actor(nn.Module):
    def __init__(self, action_dim, max_action):
        super(Actor, self).__init__()

        # TODO: You'll need to change this!
        flat_size = 31968
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 8, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            
            nn.Conv2d(32, 32, 4, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            Flatten(),
            nn.Dropout(.1),
        )
        self.dense = nn.Sequential(
            nn.Linear(flat_size, 100),
            nn.ReLU(),
            nn.Linear(100, action_dim),
            nn.Tanh(),
        )
        self.max_action = max_action
        
    def forward(self, x):
        x = self.conv(x)
        x = self.dense(x)
        x = self.max_action * x
        
        return x

class Critic(nn.Module):
    def __init__(self, action_dim, max_action):
        super(Critic, self).__init__()

        # TODO: You'll need to change this!
        flat_size = 31968

        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 8, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            
            nn.Conv2d(32, 32, 4, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            Flatten(),
            nn.Dropout(.1),
        )
        
        self.dense = nn.Sequential(
            nn.Linear(flat_size + action_dim, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
        )
        
        
        self.max_action = max_action

    def forward(self, obs, action):
        x = self.conv(obs)
        combined = torch.cat([x, action], 1)
        x = self.dense(combined)
        return x

In [5]:
local_env = launch_env()
local_env = wrap_env(local_env)
local_env = ResizeWrapper(local_env)
local_env = ImgWrapper(local_env)

# actor = Actor(action_dim=2, max_action=1.0)

for _ in range(nepisodes):
    done = False
    obs = local_env.reset()
    obs = torch.from_numpy(obs)
    
    c1 = 16
    c2 = -10
    c3 = 40
    
    while not done:
        action = np.random.random(2)
#         action = actor(obs)        
        obs, r, done, info = local_env.step(action)
#         obs = torch.from_numpy(obs)

        new_r = updated_reward(local_env, c1, c2, c3)
        print(r, new_r)
 

22:02:54|gym-duckietown|graphics.py:121|create_frame_buffers(): Falling back to non-multisampled frame buffer
22:02:54|gym-duckietown|graphics.py:121|create_frame_buffers(): Falling back to non-multisampled frame buffer
22:02:54|gym-duckietown|simulator.py:550|_load_map(): loading map file "/duckietown/simulation/gym_duckietown/maps/loop_empty.yaml"
22:02:54|gym-duckietown|objmesh.py:50|__init__(): loading mesh "duckiebot.obj"
22:02:54|gym-duckietown|objmesh.py:238|_load_mtl(): loading materials from "/duckietown/simulation/gym_duckietown/meshes/duckiebot.mtl"
22:02:54|gym-duckietown|objmesh.py:50|__init__(): loading mesh "duckie.obj"
22:02:54|gym-duckietown|graphics.py:60|load_texture(): loading texture "duckie.png"
22:02:55|gym-duckietown|objmesh.py:50|__init__(): loading mesh "cone.obj"
22:02:55|gym-duckietown|graphics.py:60|load_texture(): loading texture "cone.png"
22:02:55|gym-duckietown|graphics.py:60|load_texture(): loading texture "floor_1.png"
22:02:55|gym-duckietown|graphics

22:02:55|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
22:02:55|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
22:02:55|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [3.59826268 0.         2.08721855]
22:02:55|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.5894849  0.         2.18432262]
22:02:55|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [3.60704045 0.         1.99011448]
22:02:55|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [3.48173779 0.         2.07668522]
22:02:55|gym-duckietown|simulator.py:1107|_drivable_pos(): [3.48172154 0.         1.84866924] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7f9b0d3b20f0>, 'color': array([1, 1, 1])}
22:02:55|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True

22:02:55|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [4.02468139 0.         1.75868687]
22:02:55|gym-duckietown|simulator.py:1107|_drivable_pos(): [3.49620504 0.         2.20742184] corresponds to tile at (5, 3) which is not drivable: {'coords': (5, 3), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7f9b0d3b20f0>, 'color': array([1, 1, 1])}
22:02:55|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
22:02:55|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
22:02:55|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [3.57500004 0.         2.14999713]
22:02:55|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.49620504 0.         2.20742184]
22:02:55|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [3.65379505 0.         2.09257242]
22:02:55|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [3.50609039 0.         2.05

22:03:00|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
22:03:00|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.3
22:03:00|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [2.33703727 0.         2.57700384]
22:03:00|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [2.24568806 0.         2.542922  ]
22:03:00|gym-duckietown|simulator.py:1221|_valid_pose(): r_pos: [2.42838647 0.         2.61108568]
22:03:00|gym-duckietown|simulator.py:1222|_valid_pose(): f_pos: [2.37793548 0.         2.46738479]
22:03:00|gym-duckietown|simulator.py:1107|_drivable_pos(): [2.28873639 0.         2.68406941] corresponds to tile at (3, 4) which is not drivable: {'coords': (3, 4), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7f9b0d3b20f0>, 'color': array([1, 1, 1])}
22:03:00|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True

-0.11008460519299601 -1.3100253097773256
-0.11008460519299601 -1.3100253097773256
-0.11008460519299601 -1.3100253097773256
-0.11008460519299601 -1.3100253097773256
-0.10994247135276503 -1.309930534832813
-0.10973427984279072 -1.3096736025945486
-0.10946012063490129 -1.3093688948598565
-0.10868069834177807 -1.3083779414816166
-0.10739598064478928 -1.3071133131533754


...

0.09802345749627095 -0.9480855054466125
0.03162782668833386 -0.998367426591565
-0.041472654253652985 -1.0554294016353214
-0.11536947311425272 -1.1147564254959503
-0.18265815528039953 -1.171776529719205
-0.2383574335514057 -1.2257029599233689
-0.2885882869145472 -1.2771110230251632
-0.35060516128231034 -1.3352831290504912
-0.4278085640622986 -1.401980495646213
-0.5141062457462697 -1.475270460315291
-0.6006062304346675 -1.5469749727042725
-0.6866283144299598 -1.6164533319535042
-0.772396275093381 -1.6840888369905214
-0.8638208189509775 -1.7594510355977118
-0.9684163668093299 -1.8366008781882157
-1.069777652497864 -1.9132277833540658


22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.653872162168746, 0, 2.698700868410215] angle -1.9534853844666253
22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.65006141001737, 0, 2.7083200157271885] angle -1.9424961058888182
22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.6467636963484424, 0, 2.716927804994862] angle -1.9308167712972772
22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.6436944609470374, 0, 2.7252802630377717] angle -1.9150724255074245
22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.6404202444998814, 0, 2.7347237996450855] angle -1.8940109990249452
22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.6376009429925444, 0, 2.743454789540757] angle -1.8722578111754102
22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.635095325960894, 0, 2.7519024164322765] angle -1.8460151515398595
22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.632740979039117, 0

-1.1741814426267436 -1.9865943343986683
-1.2742802208533184 -2.054266592631906
-1.3633444411305673 -2.1332624353080667
-1.4758929211987772 -2.205973066395083
-1.5739594366318288 -2.2772740153786275
-1.6883342531725043 -2.359215531895644
-1.7990017320799376 -2.4361783479433545
-1.9134648706345474 -2.511722111972456
-2.0112760388490045 -2.5883727135635084
-2.096900054569663 -2.6651763143511675
-2.1980046857744058 -2.7540623736943735
-2.2869454678310195 -2.844338145981214
-2.3783548841810234 -2.935758896435522
-2.4907679931049387 -3.0382742743786597
-2.594383080426618 -3.130593487140149
-2.7070801913281017 -3.234377055913557


22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.606457699913163, 0, 2.8529972023910126] angle -1.84331068593722
22:03:05|gym-duckietown|simulator.py:1435|_render_img(): Pos: [3.6034380323345316, 0, 2.863783224708499] angle -1.8442273849660777
22:03:05|gym-duckietown|simulator.py:1107|_drivable_pos(): [3.58561562 0.         2.92733133] corresponds to tile at (6, 5) which is not drivable: {'coords': (6, 5), 'kind': 'floor', 'angle': 0, 'drivable': False, 'texture': <simulation.gym_duckietown.graphics.Texture object at 0x7f9b0d3b20f0>, 'color': array([1, 1, 1])}
22:03:05|gym-duckietown|simulator.py:1217|_valid_pose(): Invalid pose. Collision free: True On drivable area: False
22:03:05|gym-duckietown|simulator.py:1218|_valid_pose(): safety_factor: 1.0
22:03:05|gym-duckietown|simulator.py:1219|_valid_pose(): pos: [3.60991891 0.         2.84067482]
22:03:05|gym-duckietown|simulator.py:1220|_valid_pose(): l_pos: [3.68213267 0.         2.86092757]
22:03:05|gym-duckietown|simul

-2.8321383102721227 -3.343010727251388
-2.949810957109599 -3.4468131166996456
-1000 -3.5489798475409144


In [6]:
view_results_ipython(local_env)

**Question 0: After understanding the above computed reward, experiment with the constants for each component. What type of behavior does the above reward function penalize? Is this good or bad in context of autonomous driving? Name some other issues that can arise with single-objective optimization. In addition, give three sets of constants and explain qualitatively what types of behavior each penalizes or rewards (note, you may want to use a different action policy than random)**. Place the answers to the above in `reinforcement-learning-answers.txt`




# The Reinforcement Learning Learning Code

Below we'll see a relatively naive implementation of the actor-critic training loop, which proceeds as follows: the critic is tasked with a supervised learning problem of fitting rewards acquired by the agent. Then, the policy, using policy gradients, maximizes the return according to the critic's estimate, rather than using Monte-Carlo updates.

Below, we see an implementation of `DDPGAgent`, a class which handles the networks and training loop. 

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class DDPGAgent(object):
    def __init__(self, state_dim, action_dim, max_action=1.0, learning_rate=5e-2):
        super(DDPGAgent, self).__init__()
        self.flat = False

        self.actor = Actor(action_dim, max_action).to(device)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=learning_rate)
        
        self.critic = Critic(action_dim, max_action).to(device) # CriticCNN -> Critic
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=learning_rate)

    def predict(self, state):
        assert state.shape[0] == 3
        state = torch.FloatTensor(np.expand_dims(state, axis=0)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()

    def train(self, replay_buffer, iterations, batch_size=64, discount=0.99):
        for it in range(iterations):
            # Sample replay buffer
            sample = replay_buffer.sample(batch_size, flat=self.flat)
            state = torch.FloatTensor(sample["state"]).to(device)
            action = torch.FloatTensor(sample["action"]).to(device)
            next_state = torch.FloatTensor(sample["next_state"]).to(device)
            # NOTE: why 1 - done?
            done = torch.FloatTensor(1 - sample["done"]).to(device) 
            reward = torch.FloatTensor(sample["reward"]).to(device)
            
            # Compute the target Q value
            target_action = self.actor(next_state)
            target_Q = self.critic(next_state, target_action)
            
            # TODO: - no detach is a subtle, but important bug!
            target_Q = reward + (done * discount * target_Q)
            target_Q = target_Q.detach() # we want to keep this as a constant.
            
            # Get current Q estimate
            current_Q = self.critic(state, action)

            # Compute critic loss
            critic_loss = F.mse_loss(current_Q, target_Q)

            # Optimize the critic
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

            # Compute actor loss
            actor_action = self.actor(state)
            actor_loss = -self.critic(state, actor_action).mean()
            
            # Optimize the actor
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()
            
            print(f"Iteration {it}, critic_loss: {critic_loss}, actor_loss: {actor_loss}")
            
    def save(self, filename, directory):
        torch.save(self.actor.state_dict(), '{}/{}_actor.pth'.format(directory, filename))
        torch.save(self.critic.state_dict(), '{}/{}_critic.pth'.format(directory, filename))

    def load(self, filename, directory):
        self.actor.load_state_dict(torch.load('{}/{}_actor.pth'.format(directory, filename), map_location=device))
        self.critic.load_state_dict(torch.load('{}/{}_critic.pth'.format(directory, filename), map_location=device))


You'll notice that the training loop needs a `replay_buffer` object. In value-based and actor-critic methods in deep reinforcement learning, the use of a replay buffer is crucial. In the following sections, you'll explore why this is the case, and some other stabilization techniques that are needed in order to get the above code to work. Below, you can find an implementation of the replay buffer, as well the training loop that we use to train DDPG.

In [8]:
from dataclasses import dataclass
import typing
from typing import *


class ReplayBufferEntry(NamedTuple):
    state: np.ndarray
    next_state: np.ndarray
    action: np.ndarray
    reward: float
    done: float

# Simple replay buffer
class ReplayBuffer(object):
    def __init__(self, max_size=1e6):
        self.storage = []
        self.max_size = max_size

    # Expects tuples of (state, next_state, action, reward, done)
    def add(self, state, next_state, action, reward, done):
        if len(self.storage) < self.max_size:
            self.storage.append(ReplayBufferEntry(state, next_state, action, reward, done))
        else:
            # Remove random element in the memory before adding a new one
            self.storage.pop(random.randrange(len(self.storage)))
            self.storage.append(ReplayBufferEntry(state, next_state, action, reward, done))


    def sample(self, batch_size=100, flat=True):
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        states, next_states, actions, rewards, dones = [], [], [], [], []

        for i in ind:
            state, next_state, action, reward, done = self.storage[i]

            if flat:
                states.append(np.array(state, copy=False).flatten())
                next_states.append(np.array(next_state, copy=False).flatten())
            else:
                states.append(np.array(state, copy=False))
                next_states.append(np.array(next_state, copy=False))
            actions.append(np.array(action, copy=False))
            rewards.append(np.array(reward, copy=False))
            dones.append(np.array(done, copy=False))

        # state_sample, action_sample, next_state_sample, reward_sample, done_sample
        return {
            "state": np.stack(states),
            "next_state": np.stack(next_states),
            "action": np.stack(actions),
            "reward": np.stack(rewards).reshape(-1,1),
            "done": np.stack(dones).reshape(-1,1)
        }

In [9]:
seed_ = 123
max_timesteps = 50e3
batch_size = 64
discount = 0.99
eval_freq = 5e3
file_name = 'dt-class-rl'
# start_timesteps = 10e3

replay_buffer_size = 50 * batch_size

start_timesteps = 500
episode_timesteps = 1000

expl_noise = 0.1

env_timesteps = episode_timesteps
save_models = True

In [10]:
import os
import logging
import contextlib
logger = logging.getLogger("gym-duckietown")
logging.basicConfig(level="ERROR")


@contextlib.contextmanager
def temporarily_disable_logging(logger):
    start_value = logger.disabled
    try:
        logger.disabled = True
        yield
    except UserWarning:
        pass
    finally:
        logger.disabled = start_value

with temporarily_disable_logging(logger):
    local_env = launch_env()
    # local_env = wrap_env(local_env)
    local_env = ResizeWrapper(local_env)
    local_env = ImgWrapper(local_env)

    if not os.path.exists("./pytorch_models"):
        os.makedirs("./pytorch_models")

    # Set seeds
    seedall(seed_)

    state_dim = local_env.observation_space.shape
    action_dim = local_env.action_space.shape[0]
#     print("action dim", action_dim)
    max_action = float(local_env.action_space.high[0])
    min_action = float(local_env.action_space.low[0])
    
    # Initialize policy
    policy = DDPGAgent(state_dim, action_dim, max_action)

    replay_buffer = ReplayBuffer(max_size=replay_buffer_size)

    # Evaluate untrained policy
    evaluations= [evaluate_policy(local_env, policy)]

    total_timesteps = 0
    timesteps_since_eval = 0
    episode_num = 0
    done = True
    episode_reward = None
    env_counter = 0

    while total_timesteps < max_timesteps:
        if done:
            if total_timesteps != 0:
                print(("Total T: %d Episode Num: %d Episode T: %d Reward: %f") % (
                    total_timesteps, episode_num, episode_timesteps, episode_reward))
                policy.train(replay_buffer, episode_timesteps, batch_size, discount)

            # Evaluate episode
            if timesteps_since_eval >= eval_freq:
                timesteps_since_eval %= eval_freq
                evaluations.append(evaluate_policy(local_env, policy))

                policy.save(file_name, directory="./pytorch_models")
                np.savez("./pytorch_models/{}.npz".format(file_name),evaluations)

            # Reset environment
            env_counter += 1
            obs = local_env.reset()
            done = False
            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1

        # Select action randomly or according to policy
        if total_timesteps < start_timesteps:
            print(f"timestep {total_timestep}: taking random action")
            action = local_env.action_space.sample()
        else:
            print(f"timestep {total_timestep}: using policy")
            action = policy.predict(np.array(obs))
            if expl_noise != 0:
                action = (action + np.random.normal(
                    0,
                    expl_noise,
                    size=local_env.action_space.shape[0])
                ).clip(min_action, max_action)

        # Perform action
        new_obs, reward, done, _ = local_env.step(action)

        if episode_timesteps >= env_timesteps:
            done = True
            print("DONE")

        done_bool = 0 if episode_timesteps + 1 == env_timesteps else float(done)
        print("done bool", done)
        episode_reward += reward

        # Store data in replay buffer
        replay_buffer.add(obs, new_obs, action, reward, done_bool)

        obs = new_obs

        episode_timesteps += 1
        total_timesteps += 1
        timesteps_since_eval += 1
        print("total timesteps:", total_timesteps)

    # Final evaluation
    evaluations.append(evaluate_policy(local_env, policy))

    if save_models:
        policy.save(file_name, directory="./pytorch_models")
    np.savez("./pytorch_models/{}.npz".format(file_name),evaluations)

NameError: name 'total_timestep' is not defined

# Stabilizing DDPG

As you may notice, the above model performs poorly or doesn't converge. Your job is to improve it; first in the notebook, later in the AIDO submission. This last part of the assignment consists of four sections:

**1. There are subtle, but important, bugs that have been introduced into the code above. Your job is to find them, and explain them in your `reinforcement-learning-answers.txt`. You'll want to reread the original [DQN](https://deepmind.com/research/publications/human-level-control-through-deep-reinforcement-learning) and [DDPG](https://arxiv.org/abs/1509.02971) papers in order to better understand the issue, but by answering the following subquestions (*please put the answers to these in the submission for full credit*), you'll be on the right track:**

   a) Read some literature on actor-critic methods, including the original [actor-critic](https://papers.nips.cc/paper/1786-actor-critic-algorithms.pdf) paper. What is an issue that you see related to *non-stationarity*? Define what _non-stationarity_ means in the context of machine learning and how it relates to actor-critic methods. In addition, give some hypotheses on why reinforcement learning is much more difficult (from an optimization perspective) than supervised learning, and how the answer to the previous question and this one are related.

   b) What role does the replay buffer play in off-policy reinforcement learning? It's most important parameter is `max_size` - how does changing this value (answer for both increasing and decreasing trends) qualitatively affect the training of the algorithm?

   c) **Challenge Question:** Briefly, explain how automatic differentiation works. In addition, expand on the difference between a single-element tensor (that `requires_grad`) and a scalar value as it relates to automatic differentiation; when do we want to backpropogate through a single-element tensor, and when do we not? Take a close look at the code and how losses are being backpropogated. On paper or your favorite drawing software, draw out the actor-critic architecture *as described in the code*, and label how the actor and critic losses are backpropogated. On your diagram, highlight the particular loss that will cause issues with the above code, and fix it.
   
For the next section, please pick **either** the theoretical or the practical pathway. If you don't have access to the necessary compute, for the exercise, please do the theoretical portion. 
   
_Theoretical Component_ 

**2. We discussed a case study of DQN in class. The original authors used quite a few tricks to get this to work. Detail some of the following, and explain what problem they solve in training the DQN:**

a) Target Networks

b) Annealed Learning Rates

c) Replay Buffer

d) Random Exploration Period

e) Preprocessing the Image


**3. Read about either [TD3](https://arxiv.org/abs/1802.09477) or [Soft Actor Critic](https://arxiv.org/abs/1801.01290); for your choice, summarize what problems they are addressing with the standard actor-critic formulation, and how they solve them**


_Practical Component_ 

**2. [Optional - if you have access to compute] Using your analysis from the reward engineering ablation, train two agents (after you've found the bugs in DDPG) - one with the standard, `gym-duckietown` reward, and another with the parameters of your choosing. Report each set of parameters, and describe qualitatively what type of behavior the agent produces.**

If you don't have the resources to actually train these agents, instead describe what types of behaviors each reward function might prioritize.

**3. [Optional - if you have access to compute] Using the instructions [here](http://docs.duckietown.org/DT19/AIDO/out/embodied_rl.html), use the saved policy files from this notebook and submit using the template submission provided through the AIDO submission. Report your best submission number (i.e the one you'd like to be graded) in `reinforcement-learning-answers.txt`**